In [17]:
import matplotlib.pyplot as plt
import pandas as pd
import os
import shutil
import numpy as np
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split

In [18]:
# Path to Images Dataset Images
Images_path_part1 = '/Users/darylwanji/Desktop/Computer_Vision_Capstone_Project/data/HAM10000_images_part_1'
Images_path_part2 = '/Users/darylwanji/Desktop/Computer_Vision_Capstone_Project/data/HAM10000_images_part_2'

# Path to All Images in Dataset
final_path = '/Users/darylwanji/Desktop/Computer_Vision_Capstone_Project/data/All_Images'

In [19]:

def copy_image_files(src_folder, dest_folder):
    """
    Copy image files from the source folder to the destination folder.

    Parameters:
    src_folder (str): Path to the source folder containing image files.
    dest_folder (str): Path to the destination folder where image files will be copied.

    Returns:
    None
    """
    for filename in os.listdir(src_folder):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            shutil.copy(os.path.join(src_folder, filename), dest_folder)

        
# Create the destination folder if it doesn't exist
if not os.path.exists(final_path):
    os.makedirs(final_path)

copy_image_files(Images_path_part1, final_path)
copy_image_files(Images_path_part2, final_path)

In [20]:
meta_data_df = pd.read_csv('/Users/darylwanji/Desktop/Computer_Vision_Capstone_Project/data/Updated_metadata.csv')
meta_data_df.sample(5)

,lesion_id,image_id,dx,dx_type,age,sex,localization
2333,HAM_0004765,ISIC_0025599,vasc,histo,15,male,face
7599,HAM_0004814,ISIC_0032702,nv,histo,65,male,lower extremity
6904,HAM_0003677,ISIC_0029409,nv,histo,55,male,upper extremity
7829,HAM_0006121,ISIC_0034244,nv,histo,20,female,chest
2560,HAM_0001900,ISIC_0025557,bcc,histo,80,female,chest


In [21]:
# Add Images' Path to Metadata
meta_data_df['Image_path'] = meta_data_df['image_id'].apply(lambda x: os.path.join(final_path, f"{x}.jpg"))
#Save Meta Data to new file
meta_data_df.to_csv('/Users/darylwanji/Desktop/Computer_Vision_Capstone_Project/data/Updated_metadata.csv',index=False)

In [22]:
meta_data_df

,lesion_id,image_id,dx,dx_type,age,sex,localization,Image_path
0,HAM_0000118,ISIC_0027419,bkl,histo,80,male,scalp,/Users/darylwanji/Desktop/Computer_Vision_Caps...
1,HAM_0000118,ISIC_0025030,bkl,histo,80,male,scalp,/Users/darylwanji/Desktop/Computer_Vision_Caps...
2,HAM_0002730,ISIC_0026769,bkl,histo,80,male,scalp,/Users/darylwanji/Desktop/Computer_Vision_Caps...
3,HAM_0002730,ISIC_0025661,bkl,histo,80,male,scalp,/Users/darylwanji/Desktop/Computer_Vision_Caps...
4,HAM_0001466,ISIC_0031633,bkl,histo,75,male,ear,/Users/darylwanji/Desktop/Computer_Vision_Caps...
...,...,...,...,...,...,...,...,...
10010,HAM_0002867,ISIC_0033084,akiec,histo,40,male,abdomen,/Users/darylwanji/Desktop/Computer_Vision_Caps...
10011,HAM_0002867,ISIC_0033550,akiec,histo,40,male,abdomen,/Users/darylwanji/Desktop/Computer_Vision_Caps...
10012,HAM_0002867,ISIC_0033536,akiec,histo,40,male,abdomen,/Users/darylwanji/Desktop/Computer_Vision_Caps...
10013,HAM_0000239,ISIC_0032854,akiec,histo,80,male,face,/Users/darylwanji/Desktop/Computer_Vision_Caps...
